In [1]:
%load_ext autoreload
%autoreload 2

from sanity_check.traffic_generation import *

In [2]:
scenarios: List[Scenario] = []
instances: Dict[Scenario, List[TrafficGenerator]] = {}
instance_paths: List[str] = glob(os.path.join("/home/jovyan/work/binary_files/81/ids/", "*"))

In [3]:
def parse_instances() :
    for instance_path in instance_paths:
        scenario = parse_instance_parameters(os.path.join(instance_path, "parameters.py"))
        scenarios.append(scenario)

        node_paths = glob(os.path.join(instance_path, "*.conf_traffic_generator_run.log"))
        generators: List[TrafficGenerator] = []
        for node_path in node_paths:
            node_generator = parse_traffic_generator(node_path)
            generators.append(node_generator)

        instances[scenario] = generators

parse_instances()

In [4]:
def run_stats():
    epidemic_should = 0
    epidemic_sent = 0
    epidemic_failed_run = 0
    epidemic_ok_run = 0
    epidemic_really_sent = 0

    cadr_should = 0
    cadr_sent = 0
    cadr_failed_run = 0
    cadr_ok_run = 0
    cadr_really_sent = 0
    
    for scenario in scenarios:
        for generator in instances[scenario]:
            if scenario.routing == "epidemic":
                epidemic_should = epidemic_should + generator.num_dtnd
                epidemic_sent = epidemic_sent + generator.num_sent
                epidemic_really_sent = epidemic_really_sent + generator.really_sent
                if generator.num_sent != generator.num_dtnd:
                    epidemic_failed_run = epidemic_failed_run + 1
                else:
                    epidemic_ok_run = epidemic_ok_run + 1
            else:
                cadr_should = cadr_should + generator.num_dtnd
                cadr_sent = cadr_sent + generator.num_sent
                cadr_really_sent = cadr_really_sent + generator.really_sent
                if generator.num_sent != generator.num_dtnd:
                    cadr_failed_run = cadr_failed_run + 1
                else:
                    cadr_ok_run = cadr_ok_run + 1

    print(f"Epidemic")
    print(f"  Generator should send {epidemic_sent} bundles, but actually sent {epidemic_really_sent} bundles.")
    print(f"  DTND received {epidemic_should} bundles, {epidemic_sent - epidemic_should}) missing.")
    print(f"  {epidemic_ok_run} nodes successful, {epidemic_failed_run} failed ({(epidemic_failed_run/(epidemic_failed_run + epidemic_ok_run)) * 100}%)")
    
    #print(f"CADR")
    #print(f"  Generator should send {cadr_sent} bundles, but actually sent {cadr_really_sent} bundles.")
    #print(f"  DTND received {cadr_should} bundles, {cadr_sent - cadr_should}) missing.")
    #print(f"  {cadr_ok_run} nodes successful, {cadr_failed_run} failed ({(cadr_failed_run/(cadr_failed_run + cadr_ok_run)) * 100}%)")
run_stats()

Epidemic
  Generator should send 1750 bundles, but actually sent 1493 bundles.
  DTND received 1493 bundles, 257) missing.
  11 nodes successful, 24 failed (68.57142857142857%)


In [61]:
for scenario in scenarios:
    for other_scenario in scenarios:
        if scenario.instance_id == other_scenario.instance_id:
            continue
        if scenario.matches(other_scenario):
            generators_a = instances[scenario]
            generators_b = instances[other_scenario]

            for generator in generators_a:
                if generator.num_sent != generator.really_sent:
                    print(f"Mismatch: {scenario.instance_id}/{generator.node_id}({scenario.routing})")
                    print(f"  {generator.num_sent}, {generator.really_sent}")
                if generator.really_sent != generator.num_dtnd:
                    print(f"    {generator.num_dtnd}")
                
                for other_generator in generators_b:
                    if generator == other_generator:
                        if not generator.matches(other_generator):
                            foo = generator.matches(other_generator)
                            print(
                                f"#### FAIL: Mismatch between instances {scenario.instance_id} and {other_scenario.instance_id}:"
                            )
                            print(
                                f"#### FAIL: Params: bundles_per_node: {scenario.bundles_per_node}, seed: {scenario.seed}, payload_size: {scenario.payload_size}"
                            )
                            print(f"#### FAIL: Node: {generator.node_id}")
                            print("#### FAIL: Differences:")
                            generator.difference(other_generator)
                            print("#### FAIL: -----------------------------")

In [62]:
def wait_time_diffs():
    for scenario in scenarios:
        for generator in instances[scenario]:
            for wait_time in generator.real_wait_times:
                if int(wait_time[1]) - wait_time[0] != 0:
                    print(f"Mismatch: {scenario}")

wait_time_diffs()

Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(bundles_per_node=50, seed=0, routing='epidemic', payload_size=1000000, instance_id='703')
Mismatch: Scenario(

In [63]:
for scenario in scenarios:
    for generator in instances[scenario]:
        if(generator.bundles_per_node != generator.really_sent):
            print(f"Discrepancy: Should send {generator.bundles_per_node}, did sent: {generator.really_sent}")
            print(f"Length wait times: {len(generator.wait_times)}")
            print(f"Routing: {generator.routing}, Seed: {generator.seed}, Node ID: {generator.node_id}")
            print(f"Instance: {scenario.instance_id}")
            print(f"Soll: {generator.timestamps}") 
            print(f"ist:  {generator.sent_times}")
            print("----------------------------")

Discrepancy: Should send 50, did sent: 35
Length wait times: 50
Routing: epidemic, Seed: 0, Node ID: n36
Instance: 703
Soll: [75, 132, 257, 297, 394, 427, 507, 587, 656, 761, 833, 879, 924, 1043, 1087, 1191, 1228, 1309, 1390, 1415, 1542, 1590, 1691, 1728, 1776, 1842, 1972, 1984, 2064, 2154, 2223, 2301, 2352, 2410, 2527, 2565, 2679, 2742, 2817, 2875, 2955, 2988, 3070, 3141, 3206, 3257, 3332, 3437, 3524, 3549]
ist:  [75, 133, 258, 298, 395, 428, 508, 588, 657, 762, 834, 880, 926, 1045, 1089, 1193, 1230, 1311, 1452, 1537, 1724, 1832, 1994, 2091, 2199, 2325, 2515, 2587, 2727, 2877, 3007, 3145, 3256, 3374, 3551]
----------------------------
Discrepancy: Should send 50, did sent: 45
Length wait times: 50
Routing: epidemic, Seed: 0, Node ID: n26
Instance: 703
Soll: [96, 191, 237, 293, 376, 416, 497, 594, 651, 719, 837, 903, 976, 1020, 1073, 1179, 1258, 1309, 1354, 1441, 1533, 1581, 1657, 1738, 1800, 1843, 1944, 1991, 2069, 2158, 2227, 2277, 2349, 2414, 2533, 2568, 2640, 2706, 2828, 2856, 2963

In [64]:
start_times = {}
for scenario in scenarios:
    generator_start_times = []
    for generator in instances[scenario]:
        generator_start_times.append(generator.start_time)
    start_times[scenario.instance_id] = sorted(generator_start_times)
    
for t in start_times:
    print(start_times[t])

[1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408158, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159, 1621408159]
